In [38]:
import numpy as np
import time

In [181]:
def frac_diff(x, d):
    """
    Fractionally difference time series

    :param x: numeric vector or univariate time series
    :param d: number specifying the fractional difference order.
    :return: fractionally differenced series
    """
    if np.isnan(np.sum(x)):
        return None

    n = len(x)
    if n < 2:
        return None

    x = np.subtract(x, np.mean(x))

    # calculate weights
    weights = [0] * n
    weights[0] = -d
    for k in range(2, n):
        weights[k - 1] = weights[k - 2] * (k - 1 - d) / k

    # difference series
    ydiff = list(x)

    for i in range(0, n):
        dat = x[:i]
        w = weights[:i]
        ydiff[i] = x[i] + np.dot(w, dat[::-1])

    return ydiff

In [185]:
def frac_diff_vect(x, d):
    """
    Fractionally difference time series

    :param x: numeric vector or univariate time series
    :param d: number specifying the fractional difference order
    :return: fractionally differenced series
    """
    if np.isnan(np.sum(x)):
        return None

    n = len(x)
    if n < 2:
        return None

    x = np.subtract(x, np.mean(x))

    # calculate weights
    num = -d*np.cumprod(np.arange(1-d, n-1-d, 1))
    den = np.cumprod(np.arange(2,n,1))
    weights = np.divide(num, den)
    weights = np.insert(weights, 0, -d)
    weights = np.insert(weights, weights.shape, 0)

    # difference series
    ydiff = x.reshape(1,-1) * weights.reshape(-1,1)
    
    def tracify(k): 
        return np.trace(np.fliplr(ydiff), k)
    
    vfunc = np.vectorize(tracify)
    ydiff = x + vfunc(np.arange(x.shape[0], 0, -1))

    return ydiff

In [190]:
frac_diff_vect(np.array([1,2,3,4,-7,15, 32]), 0.5)

array([ -6.14285714,  -2.07142857,  -0.80357143,  -0.04464286,
       -11.4921875 ,  15.94921875,  23.32128906])

In [191]:
frac_diff(np.array([1,2,3,4,-7,15, 32]), 0.5)

[-6.142857142857143,
 -2.0714285714285716,
 -0.8035714285714288,
 -0.044642857142857206,
 -11.4921875,
 15.94921875,
 23.3212890625]

In [198]:
a = np.random.rand(30000)

In [199]:
start = time.time()
y = frac_diff(np.array(a), 0.05)
end = time.time()
print(end - start)

34.66284656524658


In [200]:
start = time.time()
x = frac_diff_vect(np.array(a), 0.05)
end = time.time()
print(end - start)

C:\Users\mmariantoni\AppData\Local\Temp\1\ipykernel_27280\1325045861.py:21: RuntimeWarning: divide by zero encountered in true_divide
  weights = np.divide(num, den)


8.000987768173218
